# Loading data

In [59]:
import pandas as pd 
import numpy as np

#agar mudah, letakkan file data dalam satu folder dengan file jupiter notebook nya
business_data = pd.read_csv("businesspandemi.csv")

business_data.head()

,headline,summary,headline_text
0,"Plafon KUR 2021 Naik Jadi Rp 253 T, Ini Strate...","Selain itu, pemerintah juga memutuskan untuk m...",Kementerian Koordinator Bidang Perekonomian me...
1,Buka-bukaan Pelabuhan Tanjung Priok soal Arus ...,Penurunan trafik logistik karena berbagai kegi...,Wabah COVID-19 yang secara resmi dinyatakan se...
2,"Di Tengah Pandemi, Pengusaha Restoran Ekspansi...","Pandemi COVID-19 belum usai, namun tidak pengu...","Pandemi COVID-19 belum usai, namun tidak pengu..."
3,Bagaimana Cara Holding Asuransi Jaga Keuangan ...,Indonesia Financial Group (IFG) atau BUMN Hold...,Indonesia Financial Group (IFG) atau BUMN Hold...
4,"Pandemi Nggak Ngaruh, Ibu Muda Ini Raup Cuan B...",Pandemi COVID-19 tak menyurutkan Amanda Treesh...,Pandemi COVID-19 tak menyurutkan Amanda Treesh...


# lower case

In [60]:
# ------ Case Folding --------
# gunakan fungsi Series.str.lower() pada Pandas
business_data['headline_text'] = business_data['headline_text'].str.lower()

print('Case Folding Result : \n')
print(business_data['headline_text'].head(5))

Case Folding Result : 

0    kementerian koordinator bidang perekonomian me...
1    wabah covid-19 yang secara resmi dinyatakan se...
2    pandemi covid-19 belum usai, namun tidak pengu...
3    indonesia financial group (ifg) atau bumn hold...
4    pandemi covid-19 tak menyurutkan amanda treesh...
Name: headline_text, dtype: object


# Tokenizing
Memecah teks menjadi perkata, dan membersihkan simbol-simbol

In [90]:
import string 
import re #regex library

# import word_tokenize & FreqDist from NLTK
from nltk.tokenize import word_tokenize 
from nltk.probability import FreqDist

# ------ Tokenizing ---------

def remove_tweet_special(text):
    # remove tab, new line, ans back slice
    text = text.replace('\\t'," ").replace('\\n'," ").replace('\\u'," ").replace('\\',"")
    # remove non ASCII (emoticon, chinese word, .etc)
    text = text.encode('ascii', 'replace').decode('ascii')
    # remove mention, link, hashtag
    text = ' '.join(re.sub("([@#][A-Za-z0-9]+)|(\w+:\/\/\S+)"," ", text).split())
    # remove incomplete URL
    return text.replace("http://", " ").replace("https://", " ")
                
business_data['headline_text'] = business_data['headline_text'].apply(remove_tweet_special)

#remove number
def remove_number(text):
    return  re.sub(r"\d+", "", text)

business_data['headline_text'] = business_data['headline_text'].apply(remove_number)

#remove punctuation
def remove_punctuation(text):
    return text.translate(str.maketrans("","",string.punctuation))

business_data['headline_text'] = business_data['headline_text'].apply(remove_punctuation)

#remove whitespace leading & trailing
def remove_whitespace_LT(text):
    return text.strip()

business_data['headline_text'] = business_data['headline_text'].apply(remove_whitespace_LT)

#remove multiple whitespace into single whitespace
def remove_whitespace_multiple(text):
    return re.sub('\s+',' ',text)

business_data['headline_text'] = business_data['headline_text'].apply(remove_whitespace_multiple)

# remove single char
def remove_singl_char(text):
    return re.sub(r"\b[a-zA-Z]\b", "", text)

business_data['headline_text'] = business_data['headline_text'].apply(remove_singl_char)

# NLTK word rokenize 
def word_tokenize_wrapper(text):
    return word_tokenize(text)

business_data['headline_text_tokens'] = business_data['headline_text'].apply(word_tokenize_wrapper)

print('Tokenizing Result : \n') 
print(business_data['headline_text_tokens'].head())

<>:16: DeprecationWarning: invalid escape sequence \w
<>:42: DeprecationWarning: invalid escape sequence \s
<>:16: DeprecationWarning: invalid escape sequence \w
<>:42: DeprecationWarning: invalid escape sequence \s
<ipython-input-90-9b25906f0b55>:16: DeprecationWarning: invalid escape sequence \w
  text = ' '.join(re.sub("([@#][A-Za-z0-9]+)|(\w+:\/\/\S+)"," ", text).split())
<ipython-input-90-9b25906f0b55>:42: DeprecationWarning: invalid escape sequence \s
  return re.sub('\s+',' ',text)


Tokenizing Result : 

0    [kementerian, koordinator, bidang, perekonomia...
1    [wabah, covid, yang, secara, resmi, dinyatakan...
2    [pandemi, covid, belum, usai, namun, tidak, pe...
3    [indonesia, financial, group, ifg, atau, bumn,...
4    [pandemi, covid, tak, menyurutkan, amanda, tre...
Name: headline_text_tokens, dtype: object


# NLTK calc frequency distribution
Menghitung frequency tiap kata

In [62]:
# NLTK calc frequency distribution
def freqDist_wrapper(text):
    return FreqDist(text)

business_data['headline_text_tokens_fdist'] = business_data['headline_text_tokens'].apply(freqDist_wrapper)

print('Frequency Tokens : \n') 
print(business_data['headline_text_tokens_fdist'].head().apply(lambda x : x.most_common()))

Frequency Tokens : 

0    [(kur, 5), (tahun, 4), (ini, 4), (bri, 4), (pe...
1    [(pelabuhan, 19), (yang, 14), (dan, 13), (di, ...
2    [(di, 13), (yang, 8), (pandemi, 5), (cut, 5), ...
3    [(dan, 16), (ifg, 11), (asuransi, 11), (yang, ...
4    [(di, 13), (kita, 11), (amanda, 7), (ini, 7), ...
Name: headline_text_tokens_fdist, dtype: object


# Stopwords
Membersihkan teks dari kata-kata yang termasuk ke dalam stopwords

In [64]:
from nltk.corpus import stopwords

# ----------------------- get stopword from NLTK stopword -------------------------------
# get stopword indonesia
list_stopwords = stopwords.words('bahasa')


# ---------------------------- manualy add stopword  ------------------------------------
# append additional stopword
list_stopwords.extend(["yg", "dg", "rt", "dgn", "ny", "d", 'klo', 
                       'kalo', 'amp', 'biar', 'bikin', 'bilang', 
                       'gak', 'ga', 'krn', 'nya', 'nih', 'sih', 
                       'si', 'tau', 'tdk', 'tuh', 'utk', 'ya', 
                       'jd', 'jgn', 'sdh', 'aja', 'n', 't', 
                       'nyg', 'hehe', 'pen', 'u', 'nan', 'loh', 'rt',
                       '&amp', 'yah', 'bisnis', 'pandemi', 'indonesia'])

# convert list to dictionary
list_stopwords = set(list_stopwords)

#remove stopword pada list token
def stopwords_removal(words):
    return [word for word in words if word not in list_stopwords]

business_data['headline_text_tokens_WSW'] = business_data['headline_text_tokens'].apply(stopwords_removal) 

print(business_data['headline_text_tokens_WSW'].head())

0    [kementerian, koordinator, bidang, perekonomia...
1    [wabah, covid, resmi, dinyatakan, world, healt...
2    [covid, pengusaha, restoran, membuka, peluang,...
3    [financial, group, ifg, bumn, holding, asurans...
4    [covid, menyurutkan, amanda, treeshasanti, men...
Name: headline_text_tokens_WSW, dtype: object


# Normalisasi
Mengganti kata-kata tertentu dengan kata lain yang lebih tepat

In [65]:
normalizad_word = pd.read_excel("normalisasi.xlsx")

normalizad_word_dict = {}

for index, row in normalizad_word.iterrows():
    if row[0] not in normalizad_word_dict:
        normalizad_word_dict[row[0]] = row[1] 

def normalized_term(document):
    return [normalizad_word_dict[term] if term in normalizad_word_dict else term for term in document]

business_data['headline_text_normalized'] = business_data['headline_text_tokens_WSW'].apply(normalized_term)

business_data['headline_text_normalized'].head(10)

D:\Application\Anaconda3\lib\site-packages\xlrd\xlsx.py:266: DeprecationWarning: This method will be removed in future versions.  Use 'tree.iter()' or 'list(tree.iter())' instead.
  for elem in self.tree.iter() if Element_has_iter else self.tree.getiterator():
D:\Application\Anaconda3\lib\site-packages\xlrd\xlsx.py:312: DeprecationWarning: This method will be removed in future versions.  Use 'tree.iter()' or 'list(tree.iter())' instead.
  for elem in self.tree.iter() if Element_has_iter else self.tree.getiterator():
D:\Application\Anaconda3\lib\site-packages\xlrd\xlsx.py:266: DeprecationWarning: This method will be removed in future versions.  Use 'tree.iter()' or 'list(tree.iter())' instead.
  for elem in self.tree.iter() if Element_has_iter else self.tree.getiterator():


0    [kementerian, koordinator, bidang, perekonomia...
1    [wabah, covid, resmi, dinyatakan, world, healt...
2    [covid, pengusaha, restoran, membuka, peluang,...
3    [financial, group, ifg, bumn, holding, asurans...
4    [covid, menyurutkan, amanda, treeshasanti, men...
5    [covid, mengubah, perilaku, masyarakat, restor...
6    [arif, muhammad, terkenal, content, creator, i...
7    [pt, bank, rakyat, persero, tbk, mencatatkan, ...
8    [kb, kookmin, bank, bank, nomor, korea, selata...
9    [virus, corona, memaksa, orang, ketat, mengatu...
Name: headline_text_normalized, dtype: object

# Sastrawi stemmer
Mengganti kata-kata ber imbuhan menjadi kata dasarnya

In [66]:
# import Sastrawi package
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import swifter

# create stemmer
factory = StemmerFactory()
stemmer = factory.create_stemmer()

# stemmed
def stemmed_wrapper(term):
    return stemmer.stem(term)

term_dict = {}

for document in business_data['headline_text_normalized']:
    for term in document:
        if term not in term_dict:
            term_dict[term] = ' '
            
print(len(term_dict))


10485


In [96]:
for term in term_dict:
    term_dict[term] = stemmed_wrapper(term)
    
    # untuk melihat hasilnya silahkan jalankan baris di bawah ini
    # print(term,":" ,term_dict[term])

In [98]:
# untuk melihat hasilnya silahkan jalankan baris di bawah ini
# print(term_dict)

In [69]:
# apply stemmed term to dataframe
def get_stemmed_term(document):
    return [term_dict[term] for term in document]

business_data['headline_text_tokens_stemmed'] = business_data['headline_text_normalized'].swifter.apply(get_stemmed_term)

print(business_data['headline_text_tokens_stemmed'])


0      [menteri, koordinator, bidang, ekonomi, tingka...
1      [wabah, covid, resmi, nyata, world, health, or...
2      [covid, usaha, restoran, buka, peluang, daerah...
3      [financial, group, ifg, bumn, holding, asurans...
4      [covid, surut, amanda, treeshasanti, jalan, ku...
                             ...                        
260    [online, kembang, pesat, covid, batas, ruang, ...
261    [grab, perintah, provinsi, pemprov, jawa, timu...
262    [bahas, rancang, undangundang, ruu, omnibus, l...
263    [urban, farming, tani, kota, tren, covid, masy...
264    [covid, industri, hibur, malam, paksa, tutup, ...
Name: headline_text_tokens_stemmed, Length: 265, dtype: object


In [70]:
business_data['headline_text_tokens_stemmed'][0:5]

0    [menteri, koordinator, bidang, ekonomi, tingka...
1    [wabah, covid, resmi, nyata, world, health, or...
2    [covid, usaha, restoran, buka, peluang, daerah...
3    [financial, group, ifg, bumn, holding, asurans...
4    [covid, surut, amanda, treeshasanti, jalan, ku...
Name: headline_text_tokens_stemmed, dtype: object

In [92]:
for i in range(len(business_data)):
        a=business_data.iloc[i][6]
        document.append(a)
        
document[0:5]

['covid', 'industri', 'hiburan', 'malam', 'dipaksa']

In [72]:
doc_clean = business_data['headline_text_tokens_stemmed']
doc_clean

0      [menteri, koordinator, bidang, ekonomi, tingka...
1      [wabah, covid, resmi, nyata, world, health, or...
2      [covid, usaha, restoran, buka, peluang, daerah...
3      [financial, group, ifg, bumn, holding, asurans...
4      [covid, surut, amanda, treeshasanti, jalan, ku...
                             ...                        
260    [online, kembang, pesat, covid, batas, ruang, ...
261    [grab, perintah, provinsi, pemprov, jawa, timu...
262    [bahas, rancang, undangundang, ruu, omnibus, l...
263    [urban, farming, tani, kota, tren, covid, masy...
264    [covid, industri, hibur, malam, paksa, tutup, ...
Name: headline_text_tokens_stemmed, Length: 265, dtype: object

In [73]:
import gensim
from gensim import corpora

dictionary = corpora.Dictionary(doc_clean)
print(dictionary)

doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean]

Dictionary(7715 unique tokens: ['agen', 'akibat', 'akselerasi', 'aplikasi', 'bangkit']...)


# LDA model using gensim
Proses topic modeling dengan LDA gensim

In [75]:
# Creating the object for LDA model using gensim library
Lda = gensim.models.ldamodel.LdaModel

total_topics = 5 # jumlah topik yang akan di extract
number_words = 15 # jumlah kata per topik

In [76]:
# Running and Trainign LDA model on the document term matrix.
lda_model = Lda(doc_term_matrix, num_topics=total_topics, id2word = dictionary, passes=50)

lda_model.show_topics(num_topics=total_topics, num_words=number_words)

[(0,
  '0.015*"as" + 0.008*"israel" + 0.008*"negara" + 0.007*"china" + 0.007*"kereta" + 0.006*"sepakat" + 0.006*"trump" + 0.006*"jalan" + 0.006*"terbang" + 0.005*"orang" + 0.005*"iran" + 0.005*"corona" + 0.005*"usaha" + 0.004*"covid" + 0.004*"turun"'),
 (1,
  '0.016*"usaha" + 0.013*"jual" + 0.012*"produk" + 0.009*"digital" + 0.006*"online" + 0.006*"rumah" + 0.006*"langgan" + 0.006*"milik" + 0.006*"kembang" + 0.005*"makan" + 0.005*"tingkat" + 0.005*"butuh" + 0.005*"strategi" + 0.005*"beli" + 0.005*"layan"'),
 (2,
  '0.034*"bintaro" + 0.027*"transpark" + 0.014*"huni" + 0.011*"fasilitas" + 0.010*"kawasan" + 0.008*"lift" + 0.008*"tower" + 0.006*"unjung" + 0.006*"lengkap" + 0.006*"sedia" + 0.006*"investasi" + 0.005*"wahana" + 0.005*"snow" + 0.005*"salju" + 0.005*"informasi"'),
 (3,
  '0.011*"usaha" + 0.009*"covid" + 0.006*"layan" + 0.005*"mitra" + 0.005*"milik" + 0.005*"shopeepay" + 0.005*"sehat" + 0.005*"tingkat" + 0.004*"laku" + 0.004*"umkm" + 0.004*"strategi" + 0.004*"salah" + 0.004*"has

In [77]:
# Word Count of Topic Keywords

from collections import Counter
topics = lda_model.show_topics(formatted=False)
data_flat = [w for w_list in doc_clean for w in w_list]
counter = Counter(data_flat)

out = []
for i, topic in topics:
    for word, weight in topic:
        out.append([word, i , weight, counter[word]])

df_imp_wcount = pd.DataFrame(out, columns=['word', 'topic_id', 'importance', 'word_count']) 
print(df_imp_wcount)

         word  topic_id  importance  word_count
0          as         0    0.014517         185
1      israel         0    0.008180         101
2      negara         0    0.007897         231
3       china         0    0.007122          97
4      kereta         0    0.007114         109
5     sepakat         0    0.006482          80
6       trump         0    0.006375          79
7       jalan         0    0.005815         264
8     terbang         0    0.005674          77
9       orang         0    0.005368         275
10      usaha         1    0.015679         678
11       jual         1    0.013109         309
12     produk         1    0.012436         298
13    digital         1    0.009486         306
14     online         1    0.006437         167
15      rumah         1    0.006271         230
16    langgan         1    0.006220         153
17      milik         1    0.005972         291
18    kembang         1    0.005794         271
19      makan         1    0.005338     

In [78]:
#Dominant topic and its percentage contribution in each topic
def format_topics_sentences(ldamodel=None, corpus=doc_term_matrix, texts=document):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row_list in enumerate(ldamodel[corpus]):
        row = row_list[0] if ldamodel.per_word_topics else row_list            
        # print(row)
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)

In [79]:
df_topic_sents_keywords = format_topics_sentences(ldamodel=lda_model, corpus=doc_term_matrix, texts=doc_clean)

# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']
print(df_dominant_topic.head(10))

   Document_No  Dominant_Topic  Topic_Perc_Contrib  \
0            0             4.0              0.9935   
1            1             0.0              0.7653   
2            2             0.0              0.4597   
3            3             3.0              0.9965   
4            4             1.0              0.9952   
5            5             1.0              0.5137   
6            6             4.0              0.6749   
7            7             4.0              0.6887   
8            8             3.0              0.9960   
9            9             2.0              0.6960   

                                            Keywords  \
0  ekonomi, bank, usaha, umkm, kerja, covid, ting...   
1  as, israel, negara, china, kereta, sepakat, tr...   
2  as, israel, negara, china, kereta, sepakat, tr...   
3  usaha, covid, layan, mitra, milik, shopeepay, ...   
4  usaha, jual, produk, digital, online, rumah, l...   
5  usaha, jual, produk, digital, online, rumah, l...   
6  ekonomi, b

# visualization package pyLDAvis

In [83]:
import pyLDAvis.gensim
import pickle 
import pyLDAvis
# Visualize the topics
pyLDAvis.enable_notebook()

In [84]:
import os
LDAvis_data_filepath = os.path.join('ldavis_prepared_'+str(total_topics))

In [85]:
corpus = [dictionary.doc2bow(text) for text in doc_clean]

In [86]:
# proses ini mungkin agak lama
if 1 == 1:
    LDAvis_prepared = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary)
    with open(LDAvis_data_filepath, 'wb') as f:
        pickle.dump(LDAvis_prepared, f)

In [87]:
# load the pre-prepared pyLDAvis data from disk
with open(LDAvis_data_filepath, 'rb') as f:
    LDAvis_prepared = pickle.load(f)

In [88]:
pyLDAvis.save_html(LDAvis_prepared, 'ldavis_prepared_'+ str(total_topics) +'.html')

In [89]:
# proses ini mungkin agak lama
LDAvis_prepared

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4      0.102515  0.039088       1        1  29.704439
1      0.058719  0.088944       2        1  23.083166
2     -0.303629 -0.001225       3        1  18.019217
0      0.073484 -0.188821       4        1  15.316038
3      0.068911  0.062014       5        1  13.877140, topic_info=            Term        Freq       Total Category  logprob  loglift
3009     bintaro  436.000000  436.000000  Default  30.0000  30.0000
3266   transpark  350.000000  350.000000  Default  29.0000  29.0000
778         huni  184.000000  184.000000  Default  28.0000  28.0000
1064          as  163.000000  163.000000  Default  27.0000  27.0000
495    fasilitas  167.000000  167.000000  Default  26.0000  26.0000
...          ...         ...         ...      ...      ...      ...
510      kembang   32.858215  246.962769   Topic5  -5.7034  -0.0421
525   masyarakat   32.254238  235.515717   Topic5  -5.7220  -0.0132
142     industri   30.440439  213.750076   Topic5  -5.7799   0.0259
347        pasar   30.486300  222.949127   Topic5  -5.7784  -0.0147
578        butuh   29.649132  181.698090   Topic5  -5.8062   0.1620

[360 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
6671      4  0.961176  abraham
2790      1  0.911264    agung
2790      4  0.041421    agung
2449      4  0.930932   airbnb
2449      5  0.051718   airbnb
...     ...       ...      ...
224       3  0.907920    world
224       4  0.033015    world
224       5  0.049523    world
2231      1  0.954721      yoy
3285      3  0.986193      zen

[693 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 2, 3, 1, 4])

# References

https://yunusmuhammad007.medium.com/text-preprocessing-menggunakan-pandas-nltk-dan-sastrawi-untuk-large-dataset-5fb3c0a88571

https://towardsdatascience.com/end-to-end-topic-modeling-in-python-latent-dirichlet-allocation-lda-35ce4ed6b3e0

